In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from neuroner import neuromodel
from shutil import copyfile, copy
import random
import os
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

### Functions charged with formatting the data set in a format that NeuroNER can handle.

In [3]:
def to_brat_format(path, file, drop_discontinuous=True):
    temp = pd.read_csv(path+file, sep='\t', index_col=0, names=['spans', 'label', 'txt'])
    temp.index = ['T'+str(i) for i in temp.index]
    for i in range(len(temp['spans'])):
        if ';' in temp['spans'][i]:
            offsets = np.sort(np.array(temp['spans'][i].replace(';',' ').split(), dtype=int))
            subsets = zip(offsets[1:-1:2], offsets[2::2])
            if np.array([((pair[0] == pair[1]) or (pair[0]+1 == pair[1])) for pair in subsets]).all():
                temp['spans'][i] = str(offsets[0])+' '+str(offsets[-1])
                temp.drop([temp.index[i] for i in range(len(temp)) if ';' in temp.spans[i]])
    if drop_discontinuous:
        temp = temp.drop([temp.index[i] for i in range(len(temp)) if ';' in temp.spans[i]])
    temp[['label', 'spans', 'txt']].to_csv(path+file, sep='\t', header=False)

def paths_brat_format(paths):    
    for path in paths:
        for f in os.listdir(path):
            if os.path.isfile(os.path.join(path, f)) and f.endswith('.ann'):
                to_brat_format(path, f)

Just in case you need to format the dataset

In [1]:
# paths = ['./data/train/', './data/valid/']
# paths_brat_format(paths)

## Instantiating and training the NeuroNER model
All parameters are taken from the file "parameters.ini". These parameters can be modified when a NeuroNER model is instantiated.

In [4]:
nn = neuromodel.NeuroNER()

{'train_model': 1, 'use_pretrained_model': 0, 'pretrained_model_folder': '', 'output_scores': 0, 'dataset_text_folder': './data/', 'main_evaluation_mode': 'token', 'output_folder': './output', 'use_character_lstm': 1, 'character_embedding_dimension': 25, 'character_lstm_hidden_state_dimension': 25, 'token_pretrained_embedding_filepath': '', 'token_embedding_dimension': 200, 'token_lstm_hidden_state_dimension': 200, 'use_crf': 1, 'patience': 100, 'maximum_number_of_epochs': 100, 'optimizer': 'adam', 'learning_rate': 0.001, 'gradient_clipping_value': 0.0, 'dropout_rate': 0.5, 'number_of_cpu_threads': 8, 'number_of_gpus': 1, 'experiment_name': 'test', 'tagging_format': 'bioes', 'tokenizer': 'spacy', 'spacylanguage': 'es', 'remap_unknown_tokens_to_unk': 1, 'load_only_pretrained_token_embeddings': 0, 'check_for_lowercase': 1, 'check_for_digits_replaced_with_zeros': 1, 'freeze_token_embeddings': 0, 'debug': 0, 'verbose': 0, 'plot_format': 'png', 'reload_character_embeddings': 1, 'reload_char

In [5]:
nn.fit()


Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.0965    0.4606    0.1596       977
     Concept     0.3765    0.0883    0.1431      2899
   Predicate     0.0357    0.0875    0.0507       343
   Reference     0.0157    0.3435    0.0301       131

   micro avg     0.0864    0.1795    0.1167      4350
   macro avg     0.1311    0.2450    0.0959      4350
weighted avg     0.2759    0.1795    0.1361      4350

Evaluate model on the valid set
              precision    recall  f1-score   support

      Action     0.1079    0.4881    0.1767       168
     Concept     0.3548    0.0737    0.1220       448
   Predicate     0.0174    0.0435    0.0248        46
   Reference     0.0176    0.3333    0.0334        24

   micro avg     0.0878    0.1822    0.1185       686
   macro avg     0.1244    0.2346    0.0892       686
weighted avg     0.2599    0.1822    0.1258       686

Predict label

Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/004_train.txt > ./output/_2019-06-06_11-39-46-897646/004_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/004_valid.txt > ./output/_2019-06-06_11-39-46-897646/004_valid.txt_conll_evaluation.txt
The last 2 epochs have not shown improvements on the validation set.

Starting epoch 5
Training completed in 26.02 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.8814    0.9130    0.8969       977
     Concept     0.9348    0.9738    0.9539      2899
   Predicate     0.8851    0.7638    0.8200       343
   Reference     0.8583    0.8321    0.8450       131

   micro avg     0.9172    0.939

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/009_train.txt > ./output/_2019-06-06_11-39-46-897646/009_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/009_valid.txt > ./output/_2019-06-06_11-39-46-897646/009_valid.txt_conll_evaluation.txt
The last 7 epochs have not shown improvements on the validation set.

Starting epoch 10
Training completed in 26.06 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.9405    0.9877    0.9636       977
     Concept     0.9916    0.9748    0.9831      2899
   Predicate     0.9797    0.8455    0.9077       343
   Reference     0.8944    0.9695    0.9304       1

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/014_train.txt > ./output/_2019-06-06_11-39-46-897646/014_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/014_valid.txt > ./output/_2019-06-06_11-39-46-897646/014_valid.txt_conll_evaluation.txt
The last 12 epochs have not shown improvements on the validation set.

Starting epoch 15
Training completed in 26.03 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.9858    0.9959    0.9908       977
     Concept     0.9914    0.9983    0.9948      2899
   Predicate     0.9853    0.9796    0.9825       343
   Reference     1.0000    0.9695    0.9845       

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/019_train.txt > ./output/_2019-06-06_11-39-46-897646/019_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/019_valid.txt > ./output/_2019-06-06_11-39-46-897646/019_valid.txt_conll_evaluation.txt
The last 17 epochs have not shown improvements on the validation set.

Starting epoch 20
Training completed in 26.05 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     1.0000    0.9990    0.9995       977
     Concept     0.9976    0.9997    0.9986      2899
   Predicate     0.9942    0.9913    0.9927       343
   Reference     0.9924    1.0000    0.9962       

...

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/079_train.txt > ./output/_2019-06-06_11-39-46-897646/079_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/079_valid.txt > ./output/_2019-06-06_11-39-46-897646/079_valid.txt_conll_evaluation.txt
The last 77 epochs have not shown improvements on the validation set.

Starting epoch 80
Training completed in 26.08 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.9990    1.0000    0.9995       977
     Concept     0.9997    0.9993    0.9995      2899
   Predicate     0.9885    1.0000    0.9942       343
   Reference     1.0000    1.0000    1.0000       

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/084_train.txt > ./output/_2019-06-06_11-39-46-897646/084_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/084_valid.txt > ./output/_2019-06-06_11-39-46-897646/084_valid.txt_conll_evaluation.txt
The last 82 epochs have not shown improvements on the validation set.

Starting epoch 85
Training completed in 26.08 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     1.0000    1.0000    1.0000       977
     Concept     0.9997    1.0000    0.9998      2899
   Predicate     1.0000    0.9942    0.9971       343
   Reference     1.0000    1.0000    1.0000       

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/089_train.txt > ./output/_2019-06-06_11-39-46-897646/089_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/089_valid.txt > ./output/_2019-06-06_11-39-46-897646/089_valid.txt_conll_evaluation.txt
The last 87 epochs have not shown improvements on the validation set.

Starting epoch 90
Training completed in 26.02 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     1.0000    1.0000    1.0000       977
     Concept     0.9997    1.0000    0.9998      2899
   Predicate     1.0000    0.9942    0.9971       343
   Reference     1.0000    1.0000    1.0000       

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/094_train.txt > ./output/_2019-06-06_11-39-46-897646/094_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/094_valid.txt > ./output/_2019-06-06_11-39-46-897646/094_valid.txt_conll_evaluation.txt
The last 92 epochs have not shown improvements on the validation set.

Starting epoch 95
Training completed in 26.05 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     0.9990    1.0000    0.9995       977
     Concept     0.9997    1.0000    0.9998      2899
   Predicate     0.9971    0.9971    0.9971       343
   Reference     1.0000    1.0000    1.0000       

Generating plots for the train set
Generating plots for the valid set
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/099_train.txt > ./output/_2019-06-06_11-39-46-897646/099_train.txt_conll_evaluation.txt
/usr/local/lib/python3.6/dist-packages/neuroner
shell_command: perl /usr/local/lib/python3.6/dist-packages/neuroner/conlleval < ./output/_2019-06-06_11-39-46-897646/099_valid.txt > ./output/_2019-06-06_11-39-46-897646/099_valid.txt_conll_evaluation.txt
The last 97 epochs have not shown improvements on the validation set.

Starting epoch 100
Training completed in 26.07 seconds
Evaluate model on the train set
              precision    recall  f1-score   support

      Action     1.0000    1.0000    1.0000       977
     Concept     0.9997    1.0000    0.9998      2899
   Predicate     1.0000    0.9942    0.9971       343
   Reference     1.0000    1.0000    1.0000      

Once the execution has finished, the models should be placed into the output folder